#### First get the candidate model parts (Head and Feature Extractor)

In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_uri = "models:/{}/{}".format('Laion Balanced', '1')
head_model = mlflow.pytorch.load_model(model_uri)
head_model

2023/04/20 14:04:29 WARNING mlflow.pytorch: Stored model version '1.13.1' does not match installed PyTorch version '2.0.0'


DiffFeatureDetectorParamBiDirectional(
  (cls_layer): Sequential(
    (features_dropout): Dropout(p=0.31296189949335906, inplace=False)
    (linear_0): Linear(in_features=1280, out_features=184, bias=True)
    (relu_0): ReLU()
    (hidden_dropout_0): Dropout(p=0.5838650129135917, inplace=False)
    (linear_1): Linear(in_features=184, out_features=184, bias=True)
    (relu_1): ReLU()
    (hidden_dropout_1): Dropout(p=0.5838650129135917, inplace=False)
    (linear_2): Linear(in_features=184, out_features=184, bias=True)
    (relu_2): ReLU()
    (hidden_dropout_2): Dropout(p=0.5838650129135917, inplace=False)
    (linear_out): Linear(in_features=184, out_features=1, bias=True)
  )
)

In [ ]:
from drecg.training.ignite_finetune import define_model_for_tune
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_1 = define_model_for_tune('Laion Balanced', 'ViT_LAION')

2023/04/20 14:03:54 WARNING mlflow.pytorch: Stored model version '1.13.1' does not match installed PyTorch version '2.0.0'
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
from drecg.feature_extraction.distributed import VitImageFeatureExtractor

In [2]:
model = VitImageFeatureExtractor.load_pretrained()

config.json not found in HuggingFace Hub.


In [4]:
type(model.vision_model)

transformers.models.clip.modeling_clip.CLIPVisionTransformer

In [ ]:
from transformers.models.clip.modeling_clip import CLIPVisionTransformer

In [ ]:
from transformers.modeling_outputs import BaseModelOutputWithPooling, BaseModelOutput
import torch
from typing import Optional, Union, Tuple
from transformers.models.clip.modeling_clip import CLIPVisionConfig, CLIPEncoder
from torch import nn

class CLIPVisionTransformer(nn.Module):
    def __init__(self, config: CLIPVisionConfig, embeddings, pre_layrnorm):
        super().__init__()
        self.config = config
        embed_dim = config.hidden_size

        self.embeddings = embeddings
        self.pre_layrnorm = pre_layrnorm
        self.encoder = CLIPEncoder(config)
        self.post_layernorm = nn.LayerNorm(embed_dim, eps=config.layer_norm_eps)

    def forward(self,pixel_values: torch.FloatTensor) -> torch.FloatTensor:
        hidden_states = self.embeddings(pixel_values)
        hidden_states = self.pre_layrnorm(hidden_states)

        encoder_outputs = self.encoder(
            inputs_embeds=hidden_states,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        last_hidden_state = encoder_outputs[0]
        pooled_output = last_hidden_state[:, 0, :]
        pooled_output = self.post_layernorm(pooled_output)

        return pooled_output

class CLIPEncoder(nn.Module):
    """
    Transformer encoder consisting of `config.num_hidden_layers` self attention layers. Each layer is a
    [`CLIPEncoderLayer`].

    Args:
        config: CLIPConfig
    """

    def __init__(self, config: CLIPConfig):
        super().__init__()
        self.config = config
        self.layers = nn.ModuleList([CLIPEncoderLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
        self,
        inputs_embeds,
        attention_mask: Optional[torch.Tensor] = None,
        causal_attention_mask: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
    ) -> Union[Tuple, BaseModelOutput]:



        hidden_states = inputs_embeds
        for idx, encoder_layer in enumerate(self.layers):
            layer_outputs = encoder_layer(
                hidden_states,
                attention_mask,
                causal_attention_mask,
                output_attentions=output_attentions,
            )

            hidden_states = layer_outputs[0]

        return (hidden_states,)


In [3]:
import torch
class Extractor(torch.nn.Module):
    def __init__(self, vision_model):
        super().__init__()
        self.vision_model = vision_model

    def forward(self, x):
        img_a, img_b = x
        return self.vision_model(img_a), self.vision_model(img_b)

In [4]:
model2 = Extractor(model)

In [5]:
#Only model weights in GPU: 7.9 GB
#Batch of 4 images pairs forward pass: +13.5GB
#Batch of 4 images pairs backward pass: +5GB
#Fordward pass time GPU batch 4 pairs: 300 ms
#forward + Backward  pass time CPU batch 4 pairs: 13 segs + 40 segs